In [4]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime

In [ ]:
folder = Path("data/reference/census/")
shapefiles = folder.glob("**/*bg.shp")
gdf = pd.concat([
    gpd.read_file(shp)
    for shp in shapefiles
]).pipe(gpd.GeoDataFrame)
gdf.crs = 'epsg:4269'
gdf.to_file(folder / 'block_groups.shp')

In [1]:
df = gdf[['GEOID', 'geometry']]

NameError: name 'gdf' is not defined

In [4]:
def cartesian_product_simplified(left, right):
    la, lb = len(left), len(right)
    ia2, ib2 = np.broadcast_arrays(*np.ogrid[:la,:lb])

    return pd.DataFrame(
        np.column_stack([left.values[ia2.ravel()], right.values[ib2.ravel()]]))

In [1]:
joined = cartesian_product_simplified(df, df).rename(columns={0: 'geoid_orig', 1: 'geom_orig', 2: 'geoid_dest', 3: 'geom_dest'})
joined['centroid_orig'] = joined['geom_orig'].apply(lambda x: x.centroid)
joined['centroid_dest'] = joined['geom_dest'].apply(lambda x: x.centroid)
joined['dist'] = joined.apply(lambda row: row['centroid_orig'].distance(row['centroid_dest']), axis=1)
joined.head()

NameError: name 'cartesian_product_simplified' is not defined

In [ ]:
joined.to_file(folder / 'block_groups.shp')

In [ ]:
joined[['geoid_orig']]

In [5]:
folder = Path("data/reference/census/")
# shapefiles = folder.glob("**/*bg.shp")
# gdf = pd.concat([
#     gpd.read_file(shp)
#     for shp in shapefiles
# ]).pipe(gpd.GeoDataFrame)
gdf = gpd.read_file(folder / 'block_groups.shp')

In [6]:
gdf.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,34,003,015200,5,340030152005,Block Group 5,G5030,S,208193,0,+40.9068552,-073.9772851,"POLYGON ((-73.98058 40.90828, -73.97784 40.908..."
1,34,035,050902,1,340350509021,Block Group 1,G5030,S,4133335,15714,+40.5837778,-074.5303616,"POLYGON ((-74.54604 40.59195, -74.54599 40.592..."
2,34,035,051000,2,340350510002,Block Group 2,G5030,S,5278681,483591,+40.5540135,-074.5632343,"POLYGON ((-74.58318 40.55522, -74.58268 40.556..."
3,34,035,053103,1,340350531031,Block Group 1,G5030,S,1964782,0,+40.4990189,-074.4945252,"POLYGON ((-74.50473 40.49382, -74.50461 40.495..."
4,34,003,035100,3,340030351003,Block Group 3,G5030,S,601233,0,+41.0392875,-074.0218017,"POLYGON ((-74.03154 41.03800, -74.03150 41.038..."
